# Taxa de risco

In [58]:
# Data Handling and Analysis
import math
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date, timedelta
from scipy import stats 

# Plotting and Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Configurations
import warnings
warnings.filterwarnings("ignore")

# Configuração dos gráficos
from matplotlib import rc

# Estilo dos gráficos
# Referência: https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
plt.style.use('Solarize_Light2')
%config InlineBackend.figure_format = 'retina'  # Formato das imagens

# Configurações de fontes
rc('font', **{"family":"sans-serif","sans-serif":["DejaVu Sans"], "size":9})
rc('mathtext', **{"default":"regular"})

%matplotlib inline

In [19]:
# Define the date range
start_date = date.today() - timedelta(days=1825)
end_date = date.today()

# List of stocks and corresponding company names
stocks = ['POMO4.SA', 'WEGE3.SA', 'PETR4.SA', 'BTLG11.SA', 'MGLU3.SA', 'B3SA3.SA']
company_names = ["Marcopolo", "Weg", "Petrobras", "BTG logística","Magalu", "Bovespa"]
coluns = company_names.copy()

# Initialize a dictionary to store different types of price data
price_data_dict = {key: [] for key in ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']}

def merge_dataframes(data_list):
    """Merge a list of DataFrames on the 'Date' column."""
    if not data_list:
        return None
    
    merged_df = data_list[0]
    if isinstance(merged_df.columns, pd.MultiIndex):
        merged_df.columns = merged_df.columns.get_level_values(0)

    for df in data_list[1:]:
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        merged_df = pd.merge(merged_df, df, on='Date', how='outer')
    
    return merged_df

def download_and_prepare_stock_data(stock_symbol, company_name):
    """Download stock data and prepare it for processing."""
    stock_data = yf.download(stock_symbol, start=start_date, end=end_date).reset_index()
    
    for price_type in price_data_dict:
        if price_type in stock_data.columns:
            price_df = stock_data[['Date', price_type]].rename(columns={price_type: company_name})
            price_data_dict[price_type].append(price_df)

# Download and prepare data for each stock
for stock_symbol, company_name in zip(stocks, company_names):
    download_and_prepare_stock_data(stock_symbol, company_name)

# Combine all price type dataframes
df_adj_close = merge_dataframes(price_data_dict['Adj Close'])
df_close = merge_dataframes(price_data_dict['Close'])
df_high = merge_dataframes(price_data_dict['High'])
df_low = merge_dataframes(price_data_dict['Low'])
df_open = merge_dataframes(price_data_dict['Open'])
df_volume = merge_dataframes(price_data_dict['Volume'])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [20]:
df_adj_close = df_adj_close.set_index('Date')
df_adj_close.columns = df_adj_close.columns.get_level_values(0)
df_adj_close

Price,Marcopolo,Weg,Petrobras,BTG logística,Magalu,Bovespa
Date,,,,,,
2019-11-25 00:00:00+00:00,2.931275,13.557370,10.123461,80.278259,103.209900,12.978880
2019-11-26 00:00:00+00:00,2.842448,13.338404,9.939585,80.495445,100.274361,12.633825
2019-11-27 00:00:00+00:00,2.866673,13.534077,9.987257,80.293777,106.238640,12.891934
2019-11-28 00:00:00+00:00,2.866673,13.711117,10.055358,80.262764,106.238640,13.082121
2019-11-29 00:00:00+00:00,2.882823,14.172345,9.925963,80.185188,104.910645,12.965291
...,...,...,...,...,...,...
2024-11-13 00:00:00+00:00,9.320000,54.700001,36.880001,95.627213,9.260000,10.230000
2024-11-14 00:00:00+00:00,9.420000,54.349998,37.270000,96.520004,9.000000,10.270000
2024-11-18 00:00:00+00:00,9.570000,54.000000,38.200001,96.430000,9.230000,10.000000


# Retornos Anuais

In [36]:
# retornos anuais
grouped = df_adj_close.groupby(df_adj_close.index.year).agg({col: ['first', 'last'] for col in company_names})
for col in company_names:
    grouped[(col, 'Retorno')] = np.log(grouped[(col, 'last')] / grouped[(col, 'first')])*100

# Flatten MultiIndex columns for easier reading
grouped.columns = [' '.join(col).strip() for col in grouped.columns.values]
grouped = grouped.rename_axis('Ano')

log_return_columns = [col for col in grouped.columns if 'Retorno' in col]
log_returns = grouped[log_return_columns]
log_returns

,Marcopolo Retorno,Weg Retorno,Petrobras Retorno,BTG logística Retorno,Magalu Retorno,Bovespa Retorno
Ano,,,,,,
2019,21.484460,17.597702,2.876758,16.813123,7.394666,-10.589581
2020,-43.295894,77.314408,-7.996820,-8.917675,70.680797,34.221031
2021,8.059202,-11.193801,19.124077,-4.176037,-124.926413,-54.783045
2022,-0.009592,20.223715,29.937911,1.444731,-89.713026,24.094890
2023,102.070988,-1.560298,74.373998,15.521953,-18.154967,20.645921
2024,32.486419,40.102471,13.022453,1.357855,-74.090143,-34.035621


# Retorno médio anual

In [40]:
for column in log_returns.columns:
    print(f'{column}: {log_returns[column].mean().round(3)}')

Marcopolo Retorno: 20.133
Weg Retorno: 23.747
Petrobras Retorno: 21.89
BTG logística Retorno: 3.674
Magalu Retorno: -38.135
Bovespa Retorno: -3.408


# Variância do retorno

In [49]:
for column in log_returns.columns:
    print(f'{column}: {log_returns[column].var().round(3)}')

Marcopolo Retorno: 2288.598
Weg Retorno: 1009.703
Petrobras Retorno: 832.07
BTG logística Retorno: 108.704
Magalu Retorno: 5159.769
Bovespa Retorno: 1275.993


# Desvio Padrão

In [50]:
for column in log_returns.columns:
    print(f'{column}: {(log_returns[column].var()**0.5).round(3)}')

Marcopolo Retorno: 47.839
Weg Retorno: 31.776
Petrobras Retorno: 28.846
BTG logística Retorno: 10.426
Magalu Retorno: 71.832
Bovespa Retorno: 35.721


# Coeficiente de Variação

In [52]:
for column in log_returns.columns:
    mean = log_returns[column].mean()
    std_dev = log_returns[column].std()
    cv = (std_dev / mean) * 100
    print(f'{column}: {cv.round(3)}')

Marcopolo Retorno: 237.621
Weg Retorno: 133.808
Petrobras Retorno: 131.777
BTG logística Retorno: 283.781
Magalu Retorno: -188.362
Bovespa Retorno: -1048.234


## Risco Médio Anual

In [59]:
df = df_adj_close.copy()
df = df.pct_change()
df

Price,Marcopolo,Weg,Petrobras,BTG logística,Magalu,Bovespa
Date,,,,,,
2019-11-25 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-26 00:00:00+00:00,-0.030303,-0.016151,-0.018163,0.002705,-0.028442,-0.026586
2019-11-27 00:00:00+00:00,0.008523,0.014670,0.004796,-0.002505,0.059480,0.020430
2019-11-28 00:00:00+00:00,0.000000,0.013081,0.006819,-0.000386,0.000000,0.014752
2019-11-29 00:00:00+00:00,0.005634,0.033639,-0.012868,-0.000967,-0.012500,-0.008930
...,...,...,...,...,...,...
2024-11-13 00:00:00+00:00,-0.013757,-0.012992,-0.001354,-0.006390,0.003250,-0.007760
2024-11-14 00:00:00+00:00,0.010730,-0.006399,0.010575,0.009336,-0.028078,0.003910
2024-11-18 00:00:00+00:00,0.015924,-0.006440,0.024953,-0.000932,0.025556,-0.026290


In [75]:
df.std()*math.sqrt(250)

Price
Marcopolo        0.453427
Weg              0.379164
Petrobras        0.444355
BTG logística    0.170226
Magalu           0.692446
Bovespa          0.412816
dtype: float64

# Correlação

In [69]:
df.corr()

Price,Marcopolo,Weg,Petrobras,BTG logística,Magalu,Bovespa
Price,,,,,,
Marcopolo,1.000000,0.312235,0.356087,0.255759,0.340169,0.429444
Weg,0.312235,1.000000,0.321027,0.181038,0.303617,0.434647
Petrobras,0.356087,0.321027,1.000000,0.281252,0.275226,0.411550
BTG logística,0.255759,0.181038,0.281252,1.000000,0.188760,0.228945
Magalu,0.340169,0.303617,0.275226,0.188760,1.000000,0.520703
Bovespa,0.429444,0.434647,0.411550,0.228945,0.520703,1.000000


# Risco

In [83]:
pesos = np.array([0.2, 0.2, 0.2, 0.2, 0.1, 0.1])
var_potfolio = np.dot(pesos, np.dot(df.cov()*250, pesos))
round(var_potfolio,3)

np.float64(0.072)

# Votalidade

In [82]:
vot = math.sqrt(var_potfolio)
round(vot,3)

0.269

# Risco Não Sistemático

In [78]:
r_ns = var_potfolio - np.sum((df.var()*250)*pesos)
r_ns.round(3)

np.float64(-0.108)